# 📝 미션 문제지: 한국어 BERT 감성 분류

## 📜 배경 스토리
당신은 **“코딩 요정 카카라”**가 이끄는 스타트업 **“마음 번역소”**의 신입 NLP 엔지니어입니다.  
영화 리뷰 속 팬들의 감정을 실시간으로 읽어 🍿 **“팝콘 지수”**를 예측하는 첫 프로젝트에 착수했습니다.

### 목표
1. **한국어 BERT**를 파인튜닝하여 리뷰의 긍·부정을 분류한다.  
2. 사용자가 한글 문장을 입력하면 즉시 감정을 예측하는 **`predict_sentiment`** 함수를 완성한다.  

모든 과제는 **PyTorch & Hugging Face Transformers** 생태계를 기반으로 진행합니다.


## 🛠️ 과제 단계
| 단계 | 내용 | 완료 조건 |
|-----|------|-----------|
|1|환경 설정, 데이터셋(NSMC) 로드|셀 실행 결과 데이터셋 정보 출력|
|2|토큰화 및 전처리 함수 구현|`encoded` 데이터셋 생성|
|3|BERT 분류 모델 불러오기 & `Trainer` 설정|`Trainer` 인스턴스 생성|
|4|모델 학습|학습 로그 출력 & 최종 에폭 완료|
|5|검증·테스트 평가|`accuracy` 0.75 이상 달성|
|6|실시간 예측 함수 작성|임의 문장 2개 예측 결과 출력|

> **힌트:** 각 코드 블록의 `### TODO` 부분을 채우면 됩니다.  
> 전체 코드의 **~50%**는 이미 제공되었습니다.


### 1️⃣ 환경 설정 및 데이터 로드

In [1]:
# 🚨 최초 1회만 설치 (주석 해제 후 실행)
#!pip install -q transformers datasets accelerate tqdm
#!pip install -U datasets huggingface_hub fsspec
#!pip install evaluate
#!pip install -U transformers datasets


from datasets import load_dataset

# 재현성을 위해 시드 고정
import random, numpy as np, torch
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed()

# NSMC (Naver Sentiment Movie Corpus) 로드
dataset = load_dataset("nsmc")
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


### 2️⃣ 토큰화 & 전처리

In [2]:
from transformers import AutoTokenizer

checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_fn(batch):
    """문장-> 토큰 ID 변환"""
    return tokenizer(
        batch["document"],
        padding="max_length",
        truncation=True
    )


# map 함수 적용 (batched=True)
encoded = dataset.map(tokenize_fn, batched=True)
encoded = encoded.rename_column("label", "labels")

# 작은 샘플 세트(학습 5k/검증 1k/테스트 1k) 선택
small_train = encoded["train"].shuffle(seed=0).select(range(5000))
small_valid = encoded["train"].shuffle(seed=1).select(range(1000))
small_test  = encoded["test"].shuffle(seed=2).select(range(1000))

print(small_train[0])


{'id': '9746412', 'document': '이건뭐 영화도아니다 재미도없는대 비싸기만하고 짜증나게 재미없네', 'labels': 0, 'input_ids': [2, 5370, 3005, 3771, 2119, 2227, 3606, 4697, 2119, 2899, 2259, 2104, 8092, 2015, 2154, 19521, 9801, 2075, 2318, 19113, 2203, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 3️⃣ 모델 불러오기 & Trainer 설정

In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

# BERT 분류 모델
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# TrainingArguments (일부 파라미터는 채워져 있음)
args = TrainingArguments(
    output_dir="./bert-nsmc",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=42,
)


args = TrainingArguments(
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)


# 정확도 metric
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

# TODO: Trainer 인스턴스 생성
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train,
    eval_dataset=small_valid,
    compute_metrics=compute_metrics,
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 4️⃣ 모델 학습

In [10]:
# TODO: trainer.train() 호출
### TODO ###
import wandb

wandb.init(anonymous="allow")

trainer.train()

<IPython.core.display.Javascript object>

wandb: (1) Private W&B dashboard, no account required
wandb: (2) Use an existing W&B account


wandb: Enter your choice: 
wandb: Enter your choice: 1


wandb: You chose 'Private W&B dashboard, no account required'
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anony-mouse-921091980337686677 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.408800
1000,0.277100
1500,0.180600


TrainOutput(global_step=1875, training_loss=0.26171563517252605, metrics={'train_runtime': 1432.7327, 'train_samples_per_second': 10.47, 'train_steps_per_second': 1.309, 'total_flos': 3946665830400000.0, 'train_loss': 0.26171563517252605, 'epoch': 3.0})

### 5️⃣ 모델 평가

In [11]:
# TODO: trainer.evaluate() 로 테스트 세트 정확도 출력
test_metrics = trainer.evaluate(eval_dataset=small_test)
print(test_metrics)


{'eval_loss': 0.5807258486747742, 'eval_accuracy': 0.869, 'eval_runtime': 30.1636, 'eval_samples_per_second': 33.153, 'eval_steps_per_second': 4.144, 'epoch': 3.0}


### 6️⃣ 실시간 예측 함수 구현

In [16]:
id2label = {0: "부정 😞", 1: "긍정 😃"}

def predict_sentiment(sentence: str):
    """한글 문장 → 감정 예측 결과 반환"""
    # 문장 토큰화 (batch가 아니라 단일 문장, tensor로 반환)
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # GPU 사용 가능하면 GPU로 이동
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 모델 추론 (평가 모드)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        confidence, pred_label_idx = torch.max(probs, dim=1)

    label = id2label[int(pred_label_idx)]
    confidence = float(confidence)

    return {"label": label, "confidence": confidence}


# 임의 문장 테스트
for s in ["이 영화 진짜 최고다!", "시간 아깝다...","그냥 죽어","애매하네","기대만 안하면..."]:
    print(s, "->", predict_sentiment(s))


이 영화 진짜 최고다! -> {'label': '긍정 😃', 'confidence': 0.9974000453948975}
시간 아깝다... -> {'label': '부정 😞', 'confidence': 0.9993352293968201}
그냥 죽어 -> {'label': '부정 😞', 'confidence': 0.9984160661697388}
애매하네 -> {'label': '부정 😞', 'confidence': 0.9989583492279053}
기대만 안하면... -> {'label': '긍정 😃', 'confidence': 0.9021694660186768}


### 💾 추가 과제(선택): 모델 저장 & 로딩

In [ ]:
# model.save_pretrained("./bert-nsmc-best")
# tokenizer.save_pretrained("./bert-nsmc-best")


## 🎯 제출 기준
- 모든 `### TODO ###` 영역 완성
- 테스트 세트 정확도 **≥ 0.75**
- `predict_sentiment` 함수가 두 예시 문장을 올바르게 분류

### 데이터 출처
- NSMC: <https://huggingface.co/datasets/nsmc>

> 행운을 빕니다! 카카라가 🍿 팝콘 지수를 기다리고 있어요.
